In [2]:
colab = False

if colab:
    from google.colab import drive
    drive.mount("/content/drive")

In [3]:
if colab:
    !pip install pandas
    !pip install matplotlib
    !pip install rasterio
    !pip install shapely
    !pip install descartes
    !pip install pyproj
    !pip install geopandas
    !pip install keras_unet
    !pip install h5py
    
    from IPython.display import display, HTML
    js = ('<script>function ConnectButton(){ '
            'console.log("Connect pushed"); '
            'document.querySelector("#connect").click()} '
            'setInterval(ConnectButton,3000);</script>')
    display(HTML(js))

In [4]:
from datetime import datetime
import sys
import geopandas as gpd
import pandas as pd
import os
import gc
import numpy as np
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.plot import plotting_extent
from shapely.geometry import Polygon, mapping
from rasterio.mask import mask
from os.path import join
import descartes
from PIL import Image
import h5py

import fiona 
import pyproj
import time

import warnings
from importlib import reload

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import keras
from keras.utils import Sequence
from keras.optimizers import Adam
from keras import callbacks as keras_cb 
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

# see: https://github.com/karolzak/keras-unet
from keras_unet.models import satellite_unet 
from keras_unet.losses import jaccard_distance
from keras_unet.metrics import iou, iou_thresholded
from keras_unet.utils import get_augmented, reconstruct_from_patches

# from keras import backend as K

# # prevent Tensorflow memory leakage
# K.clear_session()

#warnings.simplefilter("ignore")
print(sys.executable)

if colab:
    sys.path.append("/content/drive/My Drive/app_UNOSAT")
    #sys.path.append("/content/drive/My Drive/app_UNOSAT/src")

from src import load_data as ld
from src import preprocessing as pp
from src import prepare_for_network as pfn
from src import utils as utl

Using TensorFlow backend.


/Users/jurriaan/opt/anaconda3/envs/conda_unosatenv/bin/python


In [6]:
# local paths, set your own path here
if colab:
    #path_main = '/content/drive/My Drive/app_UNOSAT'
    path_main = '/content/drive/Computers/My Macbook Pro'
else:
    path_main = '/Volumes/other/datasets_and_ML/UNOSAT_Challenge'

path_data_main = utl.make_dir(join(path_main, "data"))

#### put data in these dirs

# I choose:
# Training: Mosul, Najaf, Nasiryah
# Validation: Souleimaniye
# Test: Bagdad, Kirkouk, Samawah, Tikrit (no labels present)
path_data_local_train = join(path_data_main, 'Train_Dataset')
path_data_local_val = join(path_data_main, 'Validation_Dataset')
path_data_local_test = join(path_data_main, 'Evaluation_Dataset')

path_model =  utl.make_dir(join(path_data_main, 'model'))

# other dirs are made automatically
dir_temp_data =  utl.make_dir(join(path_data_main, 'data_temp'))
dir_temp_train = utl.make_dir(join(dir_temp_data, 'train'))
dir_temp_val = utl.make_dir(join(dir_temp_data, 'val'))
dir_temp_dev = utl.make_dir(join(dir_temp_data, 'data_dev'))
dir_temp_eval = utl.make_dir(join(dir_temp_data, 'evaluation'))
dir_temp_plots = utl.make_dir(join(dir_temp_data, 'plots_preprocessing'))

# settings preprocessing
quantile_clip_max = 0.999
size_sub_sample = (512, 512)
dtype = 'i16'
inv_stride = 2

In [ ]:



print("x_crops shape: ", str(x_crops.shape))

x_reconstructed = reconstruct_from_patches(
    img_arr=x_crops, # required - array of cropped out images
    org_img_size=(1000, 1000), # required - original size of the image
    stride=100) # use only if stride is different from patch size

print("x_reconstructed shape: ", str(x_reconstructed.shape))

plt.figure(figsize=(10,10))
plt.imshow(x_reconstructed[0])
plt.show()